In [108]:
import numpy
import math
numpy.random.seed(1)

In [3]:
def compand(v):
    v = abs(v)
    if v <= 0.0031308 :
        return v * 12.92
    else:
        return (v ** (1./2.4))*1.055-0.055

def sRGB(xyz,ilu=0):
    # Matriz de conversão XYZ -> RGB em D65. Retirada do site do Lindbloom.
    sRGBref_D65 = [[ 3.2404542, -1.5371385, -0.4985314],
           [ -0.9692660,  1.8760108,  0.0415560],
           [ 0.0556434, -0.2040259,  1.0572252]]
    # Matriz de conversão XYZ -> RGB em D50. Retirada do site do Lindbloom.
    sRGBref_D50 = [[ 3.1338561, -1.6168667, -0.4906146],
                   [-0.9787684,  1.9161415,  0.0334540],
                   [0.0719453, -0.2289914,  1.4052427]]
    sRGBref =[sRGBref_D65,sRGBref_D50]
    rgb = numpy.dot(sRGBref[ilu],xyz)
    rgb = [ compand(i) for i in rgb]
    return rgb

def sRGBclip(xyz,rgb=None,ilu=0):
    if rgb is None:
        rgb = sRGB(xyz,ilu)
    return [max(min(x, 1), 0) for x in rgb]

sRGB([0,1,0])

[1.2069714885047491, 1.3161990874993057, 0.48902810406460645]

In [4]:
a = numpy.arange(0,1,0.1)
b = numpy.arange(0,1,0.1)
c = numpy.arange(0,1,0.1)
nums = []
rgb_nums = []
for i in a:
    for j in b:
        for k in c:
            nums += [[i,j,k]]
            rgb_nums +=[sRGB([i,j,k])]

In [105]:
Input = numpy.array(nums)
Output = numpy.array(rgb_nums)
hidden = 4
layers = numpy.array([
        2 * numpy.random.random((len(Input[0]), hidden)) - 1, 
        2 * numpy.random.random((hidden, len(Output[0]))) - 1
])

N = 100
print(numpy.shape(Input))
print(numpy.shape(Output))
print(numpy.shape(layers[0]))
print(numpy.shape(layers[1]))

(1000, 3)
(1000, 3)
(3, 4)
(4, 3)


In [8]:
def sigmoid(x):
    return 1 / (1 + numpy.exp(-x))
def Dsigmoid(x):
    return x * (1 - x)

In [113]:
def question(X):
    out0 = sigmoid(numpy.dot(X,layers[0]))
    out1 = sigmoid(numpy.dot(out0,layers[1]))
    return [out0,out1]
def evaluate(x,Y):
    err1 = Y - x[1]
    lesson1 = err1*Dsigmoid(x[1])
    err0 = numpy.dot(lesson1,x[1].T)

def memorize(i,delta):
    layer[0] += numpy.dot(i.T, delta)
    
for i in range(0,N):
    answer = question(Input)
    error = evaluate(answer,Output)
    lesson = error * Dsigmoid(answer)
    memorize(Input,lesson)
#print(layers[1])
print(question([1,0,0]))
#https://medium.com/technology-invention-and-more/how-to-build-a-multi-layered-neural-network-in-python-53ec3d1d326a

TypeError: unsupported operand type(s) for -: 'int' and 'list'

In [19]:
import matplotlib.pyplot as plt
import pandas


In [74]:
inputs = numpy.random.random((50,3))
inputs = Input
#print(inputs)

dados = { 
            'XYZ': ['X: %f Y:%f Z:%f'% tuple(v) for v in inputs],
            'RGB': ['R: %f G:%f B:%f'% tuple(sRGB(v)) for v in inputs],
            'corRGB':["#%2x%2x%2x"%(tuple([int(255*i) for i in sRGBclip(v)])) for v in inputs],
            'NN': ['R: %f G:%f B:%f'% tuple(question(v)) for v in inputs],
            'corNN':["#%2x%2x%2x"%(tuple([int(255*i) for i in question(v)])) for v in inputs],
            'erro': [numpy.linalg.norm(question(v)-sRGB(v)) for v in inputs]
        }
tabela = pandas.DataFrame(data=dados)
tabela = tabela[['XYZ','RGB','corRGB','NN','corNN','erro']]

def style(v):
    return 'background-color: '+ v
styled = tabela.style.applymap(style,
                  subset=pandas.IndexSlice[['corRGB','corNN']])
styled 